In [1]:
import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn

import qiskit.tools.jupyter
%qiskit_version_table
#%qiskit_copyright

In [2]:
import numpy as np
from qiskit import QuantumCircuit, Aer, IBMQ
from qiskit.aqua import QuantumInstance
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.test import mock
from xbm import get_all_meas_circs_bits_coefs
from pauli import get_all_meas_circs_bits_coefs_pauli
from qwc import get_all_meas_circs_bits_coefs_qwc
from genc import get_all_meas_circs_bits_coefs_gc
from utils import eval_expectation
from term_grouping import QWCCommutativity, FullCommutativity
import time
import datetime
import copy
import random
import yaml
import pickle
import matplotlib.pyplot as plt

In [3]:
seed = 42
shots = 8192

nb_qubits = 3
#band_width = 3
band_width = None
nb_trial = 10
is_psi_1 = False
#is_psi_1 = True

device = 'IBMQ'
#device = 'QASM'

In [4]:
fname = 'results_real_device_kawasaki_'+str(nb_qubits)+'qubits'

if band_width is not None:
    fname = fname + '_k3'
if is_psi_1:
    fname = fname + '_psi0psi1'

if False:
    fake_device = QasmSimulator.from_backend(mock.FakeSydney())
    coupling_map = fake_device.configuration().coupling_map
    noise_model = NoiseModel.from_backend(fake_device)
else:
    noise_model, coupling_map = None, None

In [5]:
start_time = datetime.datetime.today()

In [6]:
# Rename to your config file
with open('ibmq_config.yaml') as file:
    config = yaml.safe_load(file)
    
IBMQ.save_account(config['token'], overwrite=True)
IBMQ.load_account()
provider = IBMQ.get_provider(hub=config['provider']['hub'], group=config['provider']['group'], project=config['provider']['project'])

In [7]:
quantum_instance_statevec = QuantumInstance(Aer.get_backend('statevector_simulator'), seed_simulator=seed, seed_transpiler=seed)
quantum_instance_qasm     = QuantumInstance(Aer.get_backend('qasm_simulator'), shots=shots, seed_simulator=seed, seed_transpiler=seed, coupling_map=coupling_map, noise_model=noise_model)
quantum_instance_ibmq     = QuantumInstance(provider.get_backend(config['backend']), seed_transpiler=seed, shots=shots)

In [8]:
def get_psi_A(nb_qubits, bandwidth=None):
    psi_0 = QuantumCircuit(nb_qubits)
    psi_1 = QuantumCircuit(nb_qubits)

    for i in range(nb_qubits):
        psi_0.rx(np.random.random(),i)
        psi_0.ry(np.random.random(),i)
        psi_0.rz(np.random.random(),i)
        psi_1.rx(np.random.random(),i)
        psi_1.ry(np.random.random(),i)
        psi_1.rz(np.random.random(),i)

    rand_sign = np.array(random.choices([1,-1],k=(2**nb_qubits)**2)).reshape((2**nb_qubits,2**nb_qubits))
    mat_A = np.random.random(size=(2**nb_qubits,2**nb_qubits)).astype(np.complex128) * rand_sign
    rand_sign = np.array(random.choices([1,-1],k=(2**nb_qubits)**2)).reshape((2**nb_qubits,2**nb_qubits))
    mat_A += 1.j * np.random.random(size=(2**nb_qubits,2**nb_qubits)) * rand_sign
    mat_A *= 100.

    if bandwidth is not None:
        for i in range(2**nb_qubits):
            for j in range(2**nb_qubits):
                if np.abs(i-j)>band_width:
                    mat_A[i,j] = 0.
    
    return psi_0, psi_1, mat_A

In [9]:
def experiments(psi_0, psi_1, meas, qasm=True, ibmq=True):

    p0 = quantum_instance_statevec.execute(psi_0).get_statevector()
    if psi_1 is not None:
        p1 = quantum_instance_statevec.execute(psi_1).get_statevector()
        expectation_gt = (np.conjugate(p0)*(mat_A @ p1)).sum()
    else:
        expectation_gt = (np.conjugate(p0)*(mat_A @ p0)).sum()
    print('GT      :', np.round(expectation_gt, 15),'\n')

    start = time.time()
    expectation_statevec, results = eval_expectation(copy.deepcopy(meas), quantum_instance_statevec, params_dict=None)
    end = time.time()
    print('statevec:', np.round(expectation_statevec, 15))
    print('elapsed time', np.round(time.time()-start,1), '[s]\n')
    
    res = {'GT':expectation_gt, 'statevec':expectation_statevec, 'time(statevec)':end-start}
    
    if not np.isclose(expectation_gt, expectation_statevec):
        print(expectation_gt)
        print(expectation_statevec)
        raise ValueError()

    if qasm:
        start = time.time()
        expectation_qasm, results = eval_expectation(copy.deepcopy(meas), quantum_instance_qasm, params_dict=None)
        print('QASM    :', np.round(expectation_qasm, 15))
        end = time.time()
        print('elapsed time', np.round(end-start,1), '[s]\n')
        res.update({'QASM':expectation_qasm})
        res.update({'time(QASM)':end-start})
        res.update({'results(QASM)':[result.to_dict() for result in results]})

    if ibmq:
        start = time.time()
        expectation_ibmq, results = eval_expectation(copy.deepcopy(meas), quantum_instance_ibmq, params_dict=None)
        end = time.time()
        print('IBM-Q  :', np.round(expectation_ibmq, 15))
        print('elapsed time', np.round(end-start,1), '[s]\n')
        res.update({'IBMQ':expectation_ibmq})
        res.update({'time(IBMQ)':end-start})
        res.update({'results(IBMQ)':[result.to_dict() for result in results]})
    
    return res

In [10]:
if device=='IBMQ':
    qasm, ibmq = False, True
elif device=='QASM':
    qasm, ibmq = True, False
else:
    raise NotImplementedError()

In [11]:
results = {'XBM':[],'Naive':[],'QWC':[],'GC':[]}

for trial in range(nb_trial):
    
    if is_psi_1:
        psi_0, psi_1, mat_A = get_psi_A(nb_qubits, band_width)
    else:
        psi_0, _, mat_A = get_psi_A(nb_qubits, band_width)
        psi_1 = None
    
    print('==================================================================')
    print('                             XBM, trial=',trial)
    print('==================================================================')
    start = time.time()
    meas_xbm = get_all_meas_circs_bits_coefs(psi_0, psi_1, mat_A)
    nb_circs = len(meas_xbm)
    end = time.time()

    print('# measurements =', nb_circs)
    print('elapsed time', np.round(end-start,3), '[s]\n')

    res = experiments(psi_0, psi_1, meas_xbm, qasm=qasm, ibmq=ibmq)
    res.update({'term_grouping':end-start})
    res.update({'nb_circs':len(meas_xbm)})
    results['XBM'] += [res]

    print('==================================================================')
    print('                             Naive Pauli, trial=',trial)
    print('==================================================================')
    start = time.time()
    meas_pauli = get_all_meas_circs_bits_coefs_pauli(psi_0, psi_1, mat_A, reduce=False)
    nb_circs = len(meas_pauli)
    end = time.time()

    print('# measurements =', nb_circs)
    print('elapsed time', np.round(end-start,3), '[s]\n')

    res = experiments(psi_0, psi_1, meas_pauli, qasm=qasm, ibmq=ibmq)
    res.update({'term_grouping':end-start})
    res.update({'nb_circs':len(meas_pauli)})
    results['Naive'] += [res]

    print('==================================================================')
    print('                             QWC, trial=',trial)
    print('==================================================================')
    start = time.time()
    meas_qwc   = get_all_meas_circs_bits_coefs_qwc(psi_0, psi_1, mat_A)
    nb_circs = len(meas_qwc)
    end = time.time()

    print('# measurements =', nb_circs)
    print('elapsed time', np.round(end-start,3), '[s]\n')

    res = experiments(psi_0, psi_1, meas_qwc, qasm=qasm, ibmq=ibmq)
    res.update({'term_grouping':end-start})
    res.update({'nb_circs':len(meas_qwc)})
    results['QWC'] += [res]

    print('==================================================================')
    print('                             GC, trial=',trial)
    print('==================================================================')
    start = time.time()
    meas_gc    = get_all_meas_circs_bits_coefs_gc(psi_0, psi_1, mat_A, FullCommutativity, verbose=True)
    nb_circs = len(meas_gc)
    end = time.time()

    print('# measurements =', nb_circs)
    print('elapsed time', np.round(end-start,3), '[s]\n')

    res = experiments(psi_0, psi_1, meas_gc, qasm=qasm, ibmq=ibmq)
    res.update({'term_grouping':end-start})
    res.update({'nb_circs':len(meas_gc)})
    results['GC'] += [res]

                             XBM, trial= 0
# measurements = 15
elapsed time 0.008 [s]

GT      : (-103.50037367066503-47.922914527053834j) 

statevec: (-103.50037367066508-47.92291452705387j)
elapsed time 0.2 [s]



  0%|          | 0/15 [00:00<?, ?it/s]

IBM-Q  : (-82.03165885650651-30.052955395013733j)
elapsed time 63.8 [s]

                             Naive Pauli, trial= 0
# measurements = 64
elapsed time 0.041 [s]

GT      : (-103.50037367066503-47.922914527053834j) 

statevec: (-103.50037367066508-47.92291452705388j)
elapsed time 0.5 [s]



  0%|          | 0/64 [00:00<?, ?it/s]

IBM-Q  : (-99.66693854411358-46.18873251040999j)
elapsed time 170.2 [s]

                             QWC, trial= 0
all_symbols does not include set()
# measurements = 28
elapsed time 0.189 [s]

GT      : (-103.50037367066503-47.922914527053834j) 

statevec: (-103.50037367066504-47.92291452705383j)
elapsed time 0.2 [s]



...

IBM-Q  : (-26.277228945862802-23.591030706151887j)
elapsed time 618.0 [s]

                             QWC, trial= 9
all_symbols does not include set()
# measurements = 28
elapsed time 0.298 [s]

GT      : (-19.07179389506449-19.262918926499005j) 

statevec: (-19.071793895064456-19.262918926499037j)
elapsed time 0.4 [s]



  0%|          | 0/28 [00:00<?, ?it/s]

IBM-Q  : (-23.904781279397906-24.56117609816995j)
elapsed time 105.7 [s]

                             GC, trial= 9
MEASURECIRCUIT: Generated graph for the Hamiltonian with 63 nodes.
MEASURECIRCUIT: BronKerbosch found 9 unique circuits
MEASURECIRCUIT: Elapsed time: 0.026579s
all_symbols does not include set()
MEASURECIRCUIT: Generated graph for the Hamiltonian with 63 nodes.
# measurements = 10
elapsed time 0.18 [s]

GT      : (-19.07179389506449-19.262918926499005j) 

statevec: (-19.071793895064467-19.26291892649903j)
elapsed time 0.3 [s]



  0%|          | 0/10 [00:00<?, ?it/s]

IBM-Q  : (-15.552644623793826-22.129591788425174j)
elapsed time 210.8 [s]



In [12]:
end_time = datetime.datetime.today()

print('start  :', start_time)
print('end    :', end_time)
print('elapsed:', end_time-start_time)

start  : 2021-09-20 00:23:22.933984
end    : 2021-09-25 03:52:23.557817
elapsed: 5 days, 3:29:00.623833


In [13]:
with open(fname+'.pkl', 'wb') as f:
    pickle.dump(results, f)
#with open(fname'.pkl', 'rb') as f:
#    results = pickle.load(f)